## Environment Variables Required

Make sure you have these environment variables set before running the notebook:

```bash
export API_KEY="your_api_key"
export API_SECRET="your_api_secret"
export CLIENT_ID="your_client_id"
export CONNECTION_ID="your_connection_id"
export ANNOTATION_TEMPLATE_ID="your_annotation_template_id"
export IMPORT_USERS_FROM_PROJECT_ID="source_project_id"
export EXPORT_CONNECTION_ID="your_export_connection_id" 
```


# Labellerr SDK Demo (Spare-it)

This notebook demonstrates how to use the Labellerr SDK to:
1. Initialize the client
2. Set up a connection
3. Create a dataset from a connection
4. Create a project with annotation template
5. Import users from another project
6. Upload pre-annotations with magic wand support
7. Sync data to the dataset and project
8. Create export to S3

Make sure you have the required environment variables set before running this notebook.


## Step 1: Import Required Libraries

First, we'll import all the necessary modules from the Labellerr SDK and set up logging for debugging purposes.


In [ ]:
import os
from labellerr.core.connectors import LabellerrConnection
from labellerr.client import LabellerrClient
from labellerr.core.datasets import create_dataset_from_connection
from labellerr.core.schemas import DatasetConfig, RotationConfig
from labellerr.core.projects import create_project
from labellerr.core.schemas import CreateProjectParams
from labellerr.core.annotation_templates import LabellerrAnnotationTemplate
from labellerr.core.projects import LabellerrProject
from labellerr.core.schemas import CreateExportParams, ExportDestination

import logging
logging.basicConfig(level=logging.DEBUG)


## Step 2: Initialize Labellerr Client

Create a client instance using your API credentials. Make sure you have the following environment variables set:
- `API_KEY`: Your Labellerr API key
- `API_SECRET`: Your Labellerr API secret
- `CLIENT_ID`: Your Labellerr client ID


In [ ]:
client = LabellerrClient(
    api_key=os.getenv("API_KEY"),
    api_secret=os.getenv("API_SECRET"),
    client_id=os.getenv("CLIENT_ID"),
)

print("Client initialized successfully")


## Step 3: Set Up Connection

Create a connection object using your connection ID. This connection will be used to access your data source (in this case, an S3 bucket).

Make sure you have the `CONNECTION_ID` environment variable set.


In [ ]:
# Get connection ID from environment variables
connection_id = os.getenv("CONNECTION_ID")

connection = LabellerrConnection(
    client=client,
    connection_id=connection_id,
)

print(f"Connection established with ID: {connection_id}")


## Step 4: Configure and Create Dataset

Define the dataset configuration and create a dataset from the connection. This will import images from the specified S3 path into Labellerr.


In [ ]:
# Configure dataset parameters
dataset_config = DatasetConfig(
    dataset_name="Spareit Dataset Demo Week 76",
    dataset_description="Spareit Dataset Demo",
    data_type="image",
)

# Create dataset from S3 connection
dataset = create_dataset_from_connection(
    client=client,
    dataset_config=dataset_config,
    connection=connection_id,
    path="s3://siv1-etlimg-obj-annotations/WeeklyExports/Week_76(from_2025-11-03_to_2025-11-09)/images/",
)

print(f"Dataset created successfully: {dataset.dataset_id}")


## Step 5: Check Dataset Status

Monitor the dataset creation process by checking its status. The dataset needs to be fully processed before it can be used in a project.


In [ ]:
# Check dataset status
dataset.status()
print(f"Dataset status: {dataset.status_code}")


## Step 6: Configure Project Parameters

Set up the project configuration including rotation settings for annotation, review, and client review phases.


In [ ]:
# Configure project parameters
project_config = CreateProjectParams(
    project_name="Spareit Project Demo Week 76",
    project_description="Spareit Project Demo Week 76",
    data_type="image",
    rotations=RotationConfig(
        annotation_rotation_count=1,
        review_rotation_count=1,
        client_review_rotation_count=1,
    ),
)

print("Project configuration set up successfully")


## Step 7: Set Up Annotation Template

Load an existing annotation template that will define the annotation schema for your project.

Make sure you have the `ANNOTATION_TEMPLATE_ID` environment variable set.


In [ ]:
# Load annotation template
annotation_template = LabellerrAnnotationTemplate(
    client=client,
    annotation_template_id=os.getenv("ANNOTATION_TEMPLATE_ID"),
)

print(f"Annotation template loaded: {os.getenv('ANNOTATION_TEMPLATE_ID')}")


## Step 8: Create Project

Create the project using the configured parameters, dataset, and annotation template.


In [ ]:
# Create project
project = create_project(
    client=client,
    params=project_config,
    datasets=[dataset],
    annotation_template=annotation_template,
)

print(f"Project created successfully: {project.project_id}")


## Step 9: Check Project Status

Verify that the project was created successfully by checking its status.


In [ ]:
# Check project status
project.status()
print(f"Project status: {project.status_code}")


## Step 10: Import Users from Another Project

Import users from an existing project to the newly created project. This helps maintain consistent team access across projects.

Make sure you have the `IMPORT_USERS_FROM_PROJECT_ID` environment variable set.


In [ ]:
# Import users from another project
source_project = LabellerrProject(
    client=client, 
    project_id=os.getenv("IMPORT_USERS_FROM_PROJECT_ID")
)

project.import_users(from_project=source_project)
print("Users imported successfully")


## Step 11: Upload Pre-annotations

Upload pre-annotations to the project using a COCO JSON format file. This can help speed up the annotation process by providing initial annotations that annotators can review and refine.

The `conf_bucket` parameter determines the confidence level for the pre-annotations ("high", "medium", or "low").


In [ ]:
# Upload pre-annotations from COCO JSON file
project.upload_preannotations(
    annotation_format="coco_json", 
    annotation_file="week76.json", 
    conf_bucket="high"
)

print("Pre-annotations with magic wand uploaded successfully")


## Step 12: Sync Dataset with Connection

Synchronize the dataset with the connection to ensure any new files added to the S3 bucket are reflected in the project. This is useful for incremental data updates.


In [ ]:
# Sync dataset with S3 connection
dataset.sync_with_connection(
    project=project, 
    path="s3://siv1-etlimg-obj-annotations/WeeklyExports/Week_76(from_2025-11-03_to_2025-11-09)/images/", 
    data_type="image", 
    connection=connection
)

print("Dataset synced successfully")


## Step 13: Set Up Export Connection (Optional)

Configure an export connection to automatically export annotations to an S3 bucket. This step shows how to set up the export connection and configuration.

Make sure you have the `EXPORT_CONNECTION_ID` environment variable set if you want to use this feature.


In [ ]:
# Set up export connection
export_connection_id = os.getenv("EXPORT_CONNECTION_ID")
export_connection = LabellerrConnection(
    client=client,
    connection_id=export_connection_id,
)

print(f"Export connection established with ID: {export_connection_id}")


## Step 14: Create Export (Optional)

Create an export configuration to automatically export annotations in COCO JSON format to your S3 bucket. You can specify which annotation statuses to include in the export.

**Note:** This step is commented out in the original demo. Uncomment it when you're ready to create the export.


In [ ]:
# Create export configuration (commented out - uncomment when ready to use)
"""
project.create_export(export_config=CreateExportParams(
    export_name="Spareit Export Week 76",
    export_description="Spareit Export Week 76",
    export_format="spare_it_json",
    statuses=["client_review", "accepted"],
    export_destination=ExportDestination.S3,
    connection_id=export_connection_id,
    export_folder_path="s3://siv1-etlimg-obj-annotations/WeeklyExports/Week_76(from_2025-11-03_to_2025-11-09)/exports/",
))
"""



## Summary

This notebook demonstrated the complete workflow for:

1. ✅ Initializing the Labellerr SDK client
2. ✅ Setting up a data connection
3. ✅ Creating a dataset from an S3 connection
4. ✅ Configuring and creating a project
5. ✅ Setting up annotation templates
6. ✅ Importing users from existing projects
7. ✅ Uploading pre-annotations to speed up annotation work
8. ✅ Syncing datasets with S3 connections for incremental updates
9. ✅ Configuring export connections and exports

Your project is now ready for annotation work! You can access it through the Labellerr web interface using the project ID that was displayed above.
